In [1]:
from glob import glob
import cv2

from tqdm import tqdm

import math
from IPython.display import *
import random

import os
import multiprocessing
from multiprocessing.dummy import Pool
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from collections import Counter
from pprint import pprint

In [2]:
def show(img):
    cv2.imwrite('test.jpg', img)
    display(Image('test.jpg'))

In [3]:
labels = [x.split('/')[-2] for x in glob('/home/ypw/data/xuelang/xuelang_round1_train*/*/*.jpg')]
counter = Counter(labels)
counter = counter.most_common()

classes = [x[0] for x in counter if x[1] > 30]
classes.append('其他')
pprint(classes)

['正常', '吊经', '擦洞', '跳花', '毛洞', '织稀', '扎洞', '缺经', '毛斑', '其他']


In [4]:
!rm -rf crop_train crop_valid crop_valid_64

In [5]:
for c in classes:
    os.makedirs(f'crop_train/{c}')
    os.makedirs(f'crop_valid/{c}')
    os.makedirs(f'crop_valid_64/{c}')

In [6]:
def crop_normal(fname, path, size, stride):
    img = cv2.imread(fname)
    height, width, _ = img.shape
    
    for x1 in range(0, width - size + stride, stride):
        for y1 in range(0, height - size + stride, stride):
            x2 = x1 + size
            y2 = y1 + size
            
            if x1 < 0:
                x1 = 0
                x2 = size

            if x2 > width:
                x1 = width - size
                x2 = width

            if y1 < 0:
                y1 = 0
                y2 = size

            if y2 > height:
                y1 = height - size
                y2 = height
                
            crop_img = img[y1:y2, x1:x2]
            img_fname = fname[fname.rfind('/')+1:].replace('.jpg', '')
            cv2.imwrite(f'{path}/{img_fname}_{x1}_{y1}.png', crop_img)

In [7]:
def crop_abnormal(fname, path, size, stride):
    img = cv2.imread(fname)
    height, width, _ = img.shape
    half_size = int(size/2)

    xml_fname = fname.replace('.jpg', '.xml')
    tree = ET.parse(xml_fname)
    img_fname = fname[fname.rfind('/')+1:].replace('.jpg', '')
    img = cv2.imread(fname)

    img_vis = img.copy()

    for i, ob in enumerate(tree.findall('object')):

        bbox = ob.find('bndbox')
        tagname = ob.find('name').text

        x1, y1, x2, y2 = [int(x.text) for x in bbox.getchildren()]
        cv2.rectangle(img_vis, (x1, y1), (x2, y2), color=(0, 0, 255), thickness=3)

        x_len = x2 - x1
        y_len = y2 - y1

        center_x = int((x2 - x1)/2 + x1)
        center_y = int((y2 - y1)/2 + y1)

        cv2.circle(img_vis, (center_x,center_y), color=(0, 0, 255),radius=5, thickness=5) 

        # n单方向可切割个数，实际切割个数是2n+1
        x_n = math.ceil((x_len/2 - half_size)/size)
        y_n = math.ceil((y_len/2 - half_size)/size)

        x1_crop_init = center_x - half_size - x_n*size
        y1_crop_init = center_y - half_size - y_n*size

        for j in range(2*x_n+1):
            x1_crop = x1_crop_init + j*size
            x2_crop = x1_crop + size

            for k in range(2*y_n+1):
                y1_crop = y1_crop_init + k*size 
                y2_crop = y1_crop + size

                if x1_crop < 0:
                    x1_crop = 0
                    x2_crop = size

                if x2_crop > width:
                    x1_crop = width - size
                    x2_crop = width

                if y1_crop < 0:
                    y1_crop = 0
                    y2_crop = size

                if y2_crop > height:
                    y1_crop = height - size
                    y2_crop = height

                cv2.rectangle(img_vis, (x1_crop, y1_crop), (x2_crop, y2_crop), color=(0, 255, 0), thickness=3)

                crop_img = img[y1_crop:y2_crop, x1_crop:x2_crop]
                img_fname = fname[fname.rfind('/')+1:].replace('.jpg', '')
                cv2.imwrite(f'{path}/{img_fname}_{x1_crop}_{y1_crop}.png', crop_img)

In [8]:
def f(fnames, crop_function, path, size=334, stride=64):
    # 包装函数
    def ff(fname):
        label = fname.split('/')[-2]
        if label not in classes:
            label = '其他'
        crop_function(fname, path=f'{path}/{label}', size=size, stride=stride)

    with Pool(multiprocessing.cpu_count()) as pool:
        with tqdm(pool.imap_unordered(ff, fnames), total=len(fnames)) as pbar:
            for _ in pbar:
                pass

In [9]:
fnames = glob('/home/ypw/data/xuelang/xuelang_round1_train*/*/*.jpg')
normal = glob('/home/ypw/data/xuelang/xuelang_round1_train*/正常/*.jpg')
abnormal = list(set(fnames) - set(normal))

train_normal, valid_normal = train_test_split(normal, test_size=0.1, random_state=20180726)
train_abnormal, valid_abnormal = train_test_split(abnormal, test_size=0.1, random_state=20180726)

width = 334

f(train_normal, crop_function=crop_normal, path='crop_train', size=width, stride=width)
f(train_abnormal, crop_function=crop_abnormal, path='crop_train', size=width)

f(valid_normal, crop_function=crop_normal, path='crop_valid_64', size=width, stride=64)
f(valid_abnormal, crop_function=crop_normal, path='crop_valid_64', size=width, stride=64)

f(valid_normal, crop_function=crop_normal, path='crop_valid', size=width, stride=width)
f(valid_abnormal, crop_function=crop_normal, path='crop_valid', size=width, stride=width)

100%|██████████| 71/71 [00:01<00:00, 48.50it/s]


In [10]:
n1 = len(glob('crop_train/*/*.png'))
n2 = len(glob('crop_train/正常/*.png'))
n1, n2, n1-n2, '%.2f%%'%((n1-n2)/n1*100)

(59688, 56832, 2856, '4.78%')

In [11]:
len(glob('crop_valid/*/*.png'))

9744

In [12]:
len(glob('crop_valid_64/*/*.png'))

190008